# **Implementación de un Algoritmo Genético**


Implementar un algoritmo genético que obtenga las dimensiones de una piscina rectangular, cuyo volumen es 2500 m3, para que sea recubierta por la menor cantidad de azulejos posibles. Se debe tener en cuenta:
* La superficie de cada azulejo es 0.5 m2.

Restricciones: 
* El volumen de la piscina puede variar en +/-2% (tolerancia del +/-2%: debe estar entre 2500 - 2% y 2500 + 2%).
* La longitud máxima de cada lado de la piscina es de 150m.
* La profundidad máxima es de 60m.

Objetivo: 
- Minimizar el número de azulejos para construir una piscina de 2500 m3 de volumen. 

## Librerías

In [57]:
import random
import copy
import multiprocessing 
import numpy

## Creación de la población inicial

In [58]:
def build_initial_population(pop_size, ind_length):
    return [build_individual(ind_length) for j in range(pop_size)]

  #### Inicialización aleatoria de un individuo (cadena binaria)

In [59]:
def build_individual(length):
    ind = {}
    # rango de valores de cada var indepe que puede tener cada individuo
    ind['chromosome'] = [random.randint(0, max_long) for i in range(length)] # max_long = 150; 0 a 150: en x, y, z, 150 es el valor más alto. 

    ind['fitness'] = None
    return ind

## Evaluación del individuo

In [60]:
def evaluate(pop):
    for ind in pop:
        ind['fitness'] = fitness(ind)

#### Fitness function

Definimos dos funciones que imponen restricciones:

La función constraint_volume toma un individuo y devuelve 0 si el volumen de la piscina que representa el cromosoma está dentro del rango permitido (definido por el volumen dado +/- el 2%), o la distancia entre el volumen real y el permitido, si no es así. 





In [61]:
def constraint_volume(ind):
    # si el volumen (volumen +/- tolerancia) está entre los valores aceptados devuelvo 0 --> no penalización (solución factible)
    if volume * 0.98 <= (ind['chromosome'][0] * ind['chromosome'][1] * ind['chromosome'][2]) <= volume * 1.02:
        return 0 # no penalty --> feasible solution 
    
    # Unfeasible solution: 
    # Cuando se incumple el +/- tolerancia sobre el volumen, devolvemos una penalización que sea gradual, relativa a la gravedad de
    # cómo estás siendo no factible: no es lo mismo fallar por una unidad que fallar por 200. 
    else:
        # devolvemos el valor menos el volumen deseado
        return abs(ind['chromosome'][0] * ind['chromosome'][1] * ind['chromosome'][2] - volume)

La función constraint_dimensions evalúa si los valores generados exceden los límites máximos de longitud y profundidad (max_long y max_prof, respectivamente). Si alguna de las dimensiones excede su límite máximo (el individuo viola alguna restricción de dimensiones), la función devuelve la diferencia entre el valor generado y el máximo. Si ninguna dimensión excede su límite máximo, se devuelve 0, lo que significa que el individuo cumple con todas las restricciones de dimensiones.

In [62]:
def constraint_dimensions(ind):
    dist=0
    if ind['chromosome'][0] > max_long:
        dist = ind ['chromosome'][0] - max_long
    if ind['chromosome'][1] > max_long:
        dist = ind['chromosome'][1] - max_long
    if ind['chromosome'][2] > max_prof:
        dist = ind['chromosome'][2] - max_prof
    # Avoid
    for i in range(0,3):
        if ind['chromosome'][i] < 0:
            dist = abs(ind['chromosome'][i])
    return dist

Definimos la función fitness: 

La función fitness toma un individuo y devuelve la cantidad de azulejos necesarios para cubrir la piscina que representa el cromosoma. 


En primer lugar, la función comprueba si el individuo es viable, es decir, si cumple las restricciones:


- Si el individuo no es viable, devuelve el valor de ajuste calculado como la suma de la distancia entre el volumen real y el permitido y la distancia máxima que cualquiera de las dimensiones supera el valor máximo permitido, más una cantidad adicional calculada como la cantidad máxima de azulejos que serían necesarios para cubrir una piscina con las dimensiones máximas permitidas. 


- Si el individuo es viable, la función calcula el área de cada una de las cuatro paredes de la piscina, el área de la base, la superficie total de la piscina y el número de azulejos necesarios para cubrir la superficie total de la piscina.

Finalmente, la función devuelve el número de azulejos necesarios para cubrir la superficie total de la piscina.

In [63]:
def fitness(ind):
    
    # Constraints 
    c_vol = constraint_volume(ind)
    c_dim = constraint_dimensions(ind)
    c = c_vol + c_dim

    # Unfeasible solution 
    if c > 0:
        # Offset is calculated with the worst case: x, y, z tienen los valores máximos posibles (x=150, y=150, z=60)
            # sum of areas / tile area = num max of tiles acceptable: todo lo que vaya por encima son soluciones no factibles
        offset = (4 * (max_long * max_long) + (max_long * max_prof)) / surf_tile
        return c + offset
    
    # Feasible solution
    else:
        # Compute pool area 
        area_pared_quadrada = ind['chromosome'][0] * ind['chromosome'][2] # x*z
        area_pared_rect = ind['chromosome'][1] * ind['chromosome'][2] # y*z
        area_base = ind['chromosome'][0] * ind['chromosome'][1] # x*y
        area_piscina = 2*area_pared_quadrada + 2*area_pared_rect + area_base
        # Compute num tiles
        num_tiles =  area_piscina / surf_tile 
        # return number of tiles
        return num_tiles

## Tournament selection

In [64]:
def tournament(population, pop_size, k_tournament, optimization):
    parents_selected = []
    # Seleccionar padres hasta tener el mismo número de individuos que en la población
    for i in range(0, pop_size):
        parents_selected.append(random_selection(population, k_tournament, optimization))
    return parents_selected

#### Selección aleatoria de los mejores individuos

In [65]:
def random_selection(population, k_tournament, optimization):
    best = None
    # Seleccionar k_tournament individuos aleatoriamente
    size = int(len(population) * k_tournament)
    selected = []
    for e in range(0, size):
        i = random.randint(0, len(population) - 1)
        selected.append(population[i])
    # Seleccionar el mejor del conjunto aleatorio
    if optimization == 'MIN':
        best = list(sorted(selected, reverse=False, key=lambda ind: ind['fitness']))[0]
    else:
        best = list(sorted(selected, reverse=True, key=lambda ind: ind['fitness']))[0]
    return best.copy()

## Crossover

#### Two-point crossover

In [66]:
def two_point_crossover(father, mother):
    index1 = random.randint(1, len(father['chromosome']) - 2)
    index2 = random.randint(1, len(father['chromosome']) - 2)
    if index1 > index2: index1, index2 = index2, index1
    child1 = {}
    child2 = {}
    child1['chromosome'] = father['chromosome'][:index1] + mother['chromosome'][index1:index2] + father['chromosome'][index2:]
    child2['chromosome'] = mother['chromosome'][:index1] + father['chromosome'][index1:index2] + mother['chromosome'][index2:]
    child1['fitness'] = None
    child2['fitness'] = None
    return copy.deepcopy(child1), copy.deepcopy(child2)

## Mutation

### Uniform mutation

In [67]:
def uniform_mutation(individual, prob_mutation):
    for i in range(0, len(individual['chromosome'])):
        mutate = random.random() < prob_mutation
        if mutate:
            vary = random.randint(1, max_long)
            individual['chromosome'][i] = vary
    return individual

### Gaussian mutation

In [68]:
def gaussian_mutation(individual, prob_mutation, std=10):
    for i in range(0, len(individual['chromosome'])):
        mutate = random.random() < prob_mutation
        if mutate:
            value = int(numpy.random.normal(individual['chromosome'][i], std, 1)[0])
            individual['chromosome'][i] = value
    return individual

## Remplazo de la población

#### Elitismo de la mejor solución

In [69]:
def update_population(population, new_population, optimization):
    if optimization == 'MIN':
        new_population = sorted(new_population, reverse=False, key=lambda ind: ind['fitness'])
        population = sorted(population, reverse=False, key=lambda ind: ind['fitness'])
    else:
        new_population = sorted(new_population, reverse=True, key=lambda ind: ind['fitness'])
        population = sorted(population, reverse=True, key=lambda ind: ind['fitness'])
    new_population[-1] = population[0]
    
    return new_population

## Construcción del proceso genético

In [75]:
# Parámetros
pop_size = 100
ind_length = 3 # cada uno de los lados (x, y, z) --> var independientes
max_generations = 200
k_tournament = 0.3
optimization = 'MIN' # Minimizar el número de valdosas o azulejos para construir una piscina de 2500 m3 de volumen. 
prob_crossover = 0.9
prob_mutation = 1 / ind_length

global volume, surf_tile, max_long, max_prof
volume = 2500 #[m3] Volumen piscina
surf_tile = 0.5 #[m2] Superficie del azulejo
max_long = 150 #[m] Máxima longitud de cada lado
max_prof = 60 #[m] Máxima profundidad

# Inicialización de la población
population = build_initial_population(pop_size, ind_length)

# Evaluación de los individuos
evaluate(population)

# Bucle genético (criterio de finalización)
for generation in range(0, max_generations):
    # Selección de padres
    parents_selected = tournament(population, pop_size, k_tournament, optimization)
    # Cruce de los padres
    i = 0
    new_population = []
    while i < len(parents_selected):
        if random.random() < prob_crossover:
            child1, child2 = two_point_crossover(parents_selected[i], parents_selected[i+1])
        else:
            child1 = copy.deepcopy(parents_selected[i])
            child2 = copy.deepcopy(parents_selected[i+1])
        new_population.append(child1)
        new_population.append(child2)
        i = i + 2
    #Mutación de los hijos
    for ch in new_population:
        #Elegir tipo de mutación
        #uniform_mutation(ch, prob_mutation)
        gaussian_mutation(ch, prob_mutation, std=5)
    #Evaluación de los hijos
    evaluate(new_population)
    #Remplazar la población
    population = update_population(population, new_population, optimization)          
    if optimization == 'MIN':
        best = list(sorted(population, reverse=False, key=lambda ind: ind['fitness']))[0]
    else:
        best = list(sorted(population, reverse=True, key=lambda ind: ind['fitness']))[0]
    print("Generación", generation, ": fitness", best['fitness'], " - ", best['chromosome'])

Generación 0 : fitness 198363.0  -  [2, 16, 67]
Generación 1 : fitness 198170.0  -  [2, 19, 70]
Generación 2 : fitness 198006.0  -  [2, 19, 66]
Generación 3 : fitness 198006.0  -  [2, 19, 66]
Generación 4 : fitness 198006.0  -  [2, 19, 66]
Generación 5 : fitness 198006.0  -  [2, 19, 66]
Generación 6 : fitness 198005.0  -  [2, 19, 65]
Generación 7 : fitness 198005.0  -  [2, 19, 65]
Generación 8 : fitness 198003.0  -  [2, 20, 63]
Generación 9 : fitness 198002.0  -  [2, 20, 62]
Generación 10 : fitness 198002.0  -  [2, 20, 62]
Generación 11 : fitness 198002.0  -  [2, 20, 62]
Generación 12 : fitness 198002.0  -  [2, 20, 62]
Generación 13 : fitness 198002.0  -  [2, 20, 62]
Generación 14 : fitness 5604.0  -  [2, 21, 60]
Generación 15 : fitness 5604.0  -  [2, 21, 60]
Generación 16 : fitness 5512.0  -  [2, 21, 59]
Generación 17 : fitness 5512.0  -  [2, 21, 59]
Generación 18 : fitness 5512.0  -  [2, 21, 59]
Generación 19 : fitness 5512.0  -  [2, 21, 59]
Generación 20 : fitness 5512.0  -  [2, 21,

In [88]:
vol = 22 * 16 * 7
vol

2464

In [94]:
high_margin = 2500 + 2500*0.02 # 2550
low_margin = 2500 - 2500*0.02 # 2450

if (vol <= high_margin) and (vol >= low_margin):
    print('Volume OK')
else: 
    print('Volume NO OK')


Volume OK


In [87]:
# Compute pool area 
area_pared_quadrada = 22*7  # x*z
area_pared_rect = 16*7 # y*z
area_base = 22*16 # x*y
area_piscina = 2*area_pared_quadrada + 2*area_pared_rect + area_base
# Compute num tiles
num_tiles =  area_piscina / surf_tile

print('num_tiles',num_tiles)

num_tiles 1768.0


## Discusión de resultados y Conclusiones 

El resultado muestra que la mejor solución encontrada por el algoritmo genético tras 199 generaciones tiene un valor fitness de 1768, que representa el número de azulejos necesarios para cubrir la piscina, y las dimensiones de la piscina son [22, 16, 7].

Las dimensiones de la piscina se representan como una lista de tres valores [longitud, anchura, profundidad], en metros. En este caso, la longitud (x) es de 22 metros, la anchura (y) de 16 metros y la profundidad (z) de 7 metros. Estas dimensiones dan como resultado un volumen de piscina de 2464 m3, que está dentro del margen de tolerancia de +/-2% de 2500 m3.

El valor fitness es 1768, lo que significa que la mejor solución encontrada por el algoritmo genético requiere 1768 azulejos para cubrir la superficie de la piscina.

En conclusión, el algoritmo genético ha optimizado con éxito las dimensiones de la piscina rectangular para minimizar el número de azulejos necesarios para cubrir su superficie, satisfaciendo al mismo tiempo las restricciones y limitaciones dadas.
